# Basic Machine Learning Example

###  This example reflects what a simple manual process for coming up witn an effective model for a classificaiton problem



In [1]:
# import dependencies

import numpy as np
import pandas as pd
import pickle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold, train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
import tqdm

## EVALUATE FUNCTION

Since we will be trying lots of different models, it would be nice to have a single function that will evaluate all our models and provide a standardized reporting format.

This will allow us to easily pick out the model we want to move forward with.

This function takes in a model ( pipeline ) and our train test split data. From there it simply performes predictions and generates results

In [2]:
def evaluate(pipeline, X_train, X_test, y_train, y_test):
    '''
    Evaluate a pipeline on training and test datasets
    '''    
    pipeline.fit(X_train, y_train)
    y_train_hat = pipeline.predict(X_train)
    y_test_hat = pipeline.predict(X_test)
    train_f1 = f1_score(y_train_hat, y_train)
    train_acc = accuracy_score(y_train_hat, y_train)
    test_f1 = f1_score(y_test_hat, y_test)
    test_acc = accuracy_score(y_test_hat, y_test)

    print(f"========== Predictor: {type(pipeline).__name__} ==========")
    print(f"Training result: f1: {train_f1:.3f}, acc: {train_acc:.3f}")
    print(f"Test result: f1: {test_f1:.3f}, acc: {test_acc:.3f}")
    print()


## DATA

In this case we are reading in transfusion data.  With this data we are trying to predict in an individual has given blood on Marth 2007 based on specific features.

#### The features are:
- Recency  ->   How long since the individual last gave blood
- Frequency -> How many times has the indivuaul give blood
- Monetary -> Amount of usable blood given
- Time -> How many months have they been given blood

In [8]:
# load dataset
df = pd.read_csv("giam_celebs.csv")
df.head(3)

,Unnamed: 0,Name,Sex,DOB,Year,Eye Color,Hair Color,Distinctive Features,Height,Weight,Zodiac Sign,Tattoo Body Art
0,0,Jamie Dornan,Male,5/1/1982,1982,Blue,Brown,Eyes,6.0,170.0,Taurus,No
1,1,Keira Knightley,Female,3/26/1985,1985,Brown,Brown,Eyes,5.7,119.0,Aries,No
2,2,Chris Evans,Male,6/13/1981,1981,Blue,Brown,Muscular,6.0,176.5,Gemini,Yes


In [6]:
# build X and y matrices
X = df.Year.values.reshape(-1, 1)
y = df.Weight.values.reshape(-1, 1)

print("Shape: ", X.shape, y.shape)
X

AttributeError: 'DataFrame' object has no attribute 'tc'

In [4]:
## Preliminary Data Analysis
df.isnull().sum(axis = 0)

Unnamed: 0              0
Name                    0
Sex                     0
DOB                     0
Birth Year              0
Eye Color               0
Hair Color              0
Distinctive Features    0
Height(ft)              0
Weight(lbs)             0
Zodiac Sign             0
Tattoo Body Art         0
dtype: int64

## Train Test Split

The stratify argument is used to make sure the train test split data has similar populations

In [6]:
# split to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Pick A Model For A Base Point To Evaluate Other Models Against

In this case we are choosing Logistric Regression

In [7]:
# try LogisticRegression to establish a baseline performance
pipeline = Pipeline([
    ('scale', StandardScaler()), # remember to scale first before feeding data into lgr
    ('lgr', LogisticRegression()),
])
evaluate(pipeline, X_train, X_test, y_train, y_test)

========== Predictor: Pipeline ==========
Training result: f1: 0.201, acc: 0.774
Test result: f1: 0.186, acc: 0.767



## Now Let's Try A Few More...

In [8]:
# try other predictors
evaluate(XGBClassifier(n_jobs=-1), X_train, X_test, y_train, y_test)
evaluate(LGBMClassifier(n_jobs=-1), X_train, X_test, y_train, y_test)
evaluate(RandomForestClassifier(n_jobs=-1), X_train, X_test, y_train, y_test)
evaluate(GradientBoostingClassifier(), X_train, X_test, y_train, y_test)

========== Predictor: XGBClassifier ==========
Training result: f1: 0.798, acc: 0.916
Test result: f1: 0.471, acc: 0.760

========== Predictor: LGBMClassifier ==========
Training result: f1: 0.672, acc: 0.871
Test result: f1: 0.464, acc: 0.753

========== Predictor: RandomForestClassifier ==========
Training result: f1: 0.855, acc: 0.938
Test result: f1: 0.394, acc: 0.733

========== Predictor: GradientBoostingClassifier ==========
Training result: f1: 0.597, acc: 0.851
Test result: f1: 0.467, acc: 0.787



## Let's Pick a Final Model To Move Forward With

From the above evaluations, it looks like XGBClassifier is a very promising candidate

We will then hypertune the classifier model to come up with the best model we can.

## Let's Create Our Tuning Object

In [9]:
# RandomizedSearchCV on XGB
xgb_param_grid = {
    'n_estimators': [10, 20, 50, 100, 200, 300, 400],
    'max_depth': np.arange(5, 20),
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'subsample': np.arange(0.5, 1.0, 0.05),
    'min_child_weight': np.arange(1, 10),
    'colsample_bytree': np.arange(0.2, 1.0, 0.1),
    'gamma': [0, 0.001, 0.002, 0.003, 0.004, 0.005, 1e-2],
    'n_jobs': [-1]
}


## Let's find The Best Model We Can

The RandomizedSearchCV function will try all our combinations above and select the most accurate model.  

That best model is found in the best_estimator_ property of the RandomizedSerachCV object. 

In [10]:
predictor = XGBClassifier()
rs = RandomizedSearchCV(predictor, xgb_param_grid, cv=5, scoring='f1', n_jobs=-1, n_iter=100, verbose=1)
rs.fit(X_train, y_train)
evaluate(rs.best_estimator_, X_train, X_test, y_train, y_test)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   13.1s finished


========== Predictor: XGBClassifier ==========
Training result: f1: 0.667, acc: 0.868
Test result: f1: 0.478, acc: 0.767



# Evaluate Our Model Further

Now we are going to shuffle the data over and over and apply our new model to the results to further determine if we want to use this model.

In [11]:
# evaluate model with kfold
kfold = KFold(n_splits=10)
results = cross_val_score(rs.best_estimator_, X, y, cv=kfold, n_jobs=-1)
print("Results: %.2f (%.2f) accuracy" % (results.mean(), results.std()))

Results: 0.77 (0.12) accuracy


## Save The Model For Future Use

In [12]:
# save model
with open(f'best_xgb_model.pickle', 'wb') as f:
    pickle.dump(rs.best_estimator_, f)

## Confirm File

Make sure the operating system you are NOT using is commented out below

In [13]:
# windows
! dir best_xgb*
# mac / linux / Unix
# ! ls -a best_xgb*

 Volume in drive C has no label.
 Volume Serial Number is 5075-944D

 Directory of C:\Users\GPS\Desktop\Im your CLONE!!!\extra_ml_stuff\00_Project_3_Type_Example

12/01/2020  11:29 AM           292,714 best_xgb_model.pickle
               1 File(s)        292,714 bytes
               0 Dir(s)  115,740,356,608 bytes free
